# HW1 : Data Science in Twitter Data

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](https://www.safaribooksonline.com/library/view/mining-the-social/9781491973547/?orpq) 
* The codes for Chapters 1 and 9 are in [here](https://github.com/mikhailklassen/Mining-the-Social-Web-3rd-Edition/tree/master/notebooks).

** NOTE **
* Please don't forget to save the notebook frequently when working in Jupyter Notebook, otherwise the changes you made can be lost.

*----------------------

## Data Collection: Download Twitter Data using API

* As the first step of Data Science process, let's collect some twitter data. Choose a keyword and search tweets containing the keyword by using Twitter search API, and then download the searched tweets (including the meta data that the Search API returns) in a file. It is recommended that the number of searched tweets should be at least 300.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [1]:
# What keyword did you choose?:  <sonic>

In [109]:
import io, json
from twitter import *
# from prettytable import PrettyTable
import prettytable
from textwrap import fill
from collections import Counter

In [2]:
CONSUMER_KEY = 'tVWZWJfKFv2s4qdJIOwjPBIwk'
CONSUMER_SECRET ='17B4xPaZJ5NNHwfXmnRAuVvmYkb8JnvfEXhXKGI3GuTWZ2tiTO'
OAUTH_TOKEN = '1190012319213867008-O8vhqrklw1Z1kTcchMa2t5X9Qhscch'
OAUTH_TOKEN_SECRET = 'nLyHDpan9zyCV5lTit4XKppm44QWUeW87jsHrQnl3om49'

t = Twitter(auth=OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))
print(t)


In [32]:
search_results = t.search.tweets(q="sonic", count= '100')


In [53]:
search_results.keys()

dict_keys(['statuses', 'search_metadata'])

In [36]:
search_results['statuses'][0].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [99]:
index = 2
print(search_results['statuses'][index]['entities']['hashtags'])
search_results['statuses'][index]['entities']['user_mentions']

[{'text': 'SonicMovie', 'indices': [16, 27]}]


[{'screen_name': 'isa_yfn415',
  'name': '*🍩이사Isa🌭*',
  'id': 782560333596495872,
  'id_str': '782560333596495872',
  'indices': [3, 14]}]

In [56]:
# heavily inspired from Chapter 9, Page 362 from Mining the Social Web
# source: https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
def save_json(data):
    with io.open('sonic-data.json', 'w', encoding='utf-8') as f:
        json.dump(data,f)
    
def load_json():
    with io.open('sonic-data.json', encoding='utf-8') as f:
        return json.load(f)

In [57]:
save_json(search_results)
a = load_json()
print(a.keys())

dict_keys(['statuses', 'search_metadata'])


### Report  statistics about the tweets you collected 

In [ ]:
# The total number of tweets collected:  < INSERT THE NUMBER HERE>

'''
Links:
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
https://stackoverflow.com/questions/23590688/how-to-get-more-than-100-tweets-from-the-twitter-search-or-streaming-api-using-t
https://twitter.com/search?q=sonic&src=typed_query

todo:
- increase tweet amount request count
- remove duplicated or RT when doing most popular tweets
- 
'''


# Data Exploration: Exploring the Tweets and Tweet Entities

**(1) Word Count:** 
* Load the tweets you collected in the local file (txt or json)
* compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 most-frequent words with their counts

In [74]:
# load json
results = load_json()
# dictionary that keeps track of frequency of words
words_collection = {}
# get number of tweets
number_tweets = len(results['statuses'])

# populate dictionary
for tweet_index in range(number_tweets):
    for word in results['statuses'][tweet_index]['text'].split():
        # if word not in dict then add, else increase count
        if word in words_collection.keys():
            words_collection[word] += 1
        else:
            words_collection[word] = 1

# sort dictionary based on frequency
sorted_results = {k: v for k, v in sorted(words_collection.items(), key=lambda item: item[1], reverse=True)}

# create pretty table
table = PrettyTable(field_names=['word','count'])
for index, word_count_pair in enumerate(sorted_results.items()):
    if index == 30:
        break
    table.add_row(word_count_pair)

print(table)


+--------+-------+
|  word  | count |
+--------+-------+
|   RT   |   58  |
| Sonic  |   37  |
|  the   |   37  |
|   I    |   33  |
|   to   |   24  |
|   a    |   23  |
| sonic  |   22  |
|  and   |   22  |
|   of   |   17  |
|   in   |   13  |
|   so   |   12  |
|  for   |   12  |
|   is   |   12  |
|   my   |   11  |
|  was   |   10  |
| April  |   10  |
| SONIC  |   10  |
|  The   |   10  |
|  with  |   9   |
|   on   |   9   |
| Happy  |   9   |
| movie  |   9   |
|   it   |   8   |
|  not   |   7   |
|   X!   |   6   |
|   7,   |   6   |
| SUMMER |   6   |
|  2019  |   6   |
| Replay |   6   |
|  you   |   6   |
+--------+-------+


** (2) Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 most-retweeted tweets in your collection, i.e., the tweets with the largest number of retweet counts.


In [117]:
# list that keeps track of popular tweets
popular_tweets = []
# get number of tweets
number_tweets = len(results['statuses'])

# populate list
for tweet_index in range(number_tweets):
    text = results['statuses'][tweet_index]['text']
    retweet_count = results['statuses'][tweet_index]['retweet_count']
    popular_tweets.append((text, retweet_count))

# sort list based on retweet frequency
sorted_results = sorted(popular_tweets, key=lambda item: item[1], reverse=True)

# create pretty table
table = PrettyTable(field_names=['text','retweet count'])
for index, tweet_count_pair in enumerate(sorted_results):
    if index == 10:
        break
    table.add_row((fill(tweet_count_pair[0], width = 40), tweet_count_pair[1]))
    print(f"[{index + 1}] " + tweet_count_pair[0])

table.hrules = prettytable.ALL
print(table)


[1] RT @nakeyjakey: what if linguini from ratatouille was having sex and the girl pulled his hair and he started cooking spaghetti
[2] RT @IndigoBeatss: Ok so heres my reference thing

Apparently ive been drawing teen tails this whole time cuz i keep drawing him with those…
[3] RT @RandomPocket12: I never laughed so hard with Sonic X until this, why couldn't the whole show be like this?! https://t.co/00yr5g36G6
[4] RT @isa_yfn415: #SonicMovie

I wanted to draw Sonic and Carl's first meeting!
I thought Sonic was probably happy when Carl noticed him http…
[5] RT @whoisnoahagain: The casting choice for eggman in the sonic movie was phenomenal https://t.co/bvGRuwlUp5
[6] RT @whoisnoahagain: The casting choice for eggman in the sonic movie was phenomenal https://t.co/bvGRuwlUp5
[7] RT @whoisnoahagain: The casting choice for eggman in the sonic movie was phenomenal https://t.co/bvGRuwlUp5
[8] RT @tmsanime: Happy Birthday Sonic X! Here's an exclusive clip featuring never-before-seen materials

**(3) Find the most popular Tweet Entities in your collection of tweets**

Please plot the top 10 most-frequent hashtags and top 10 most-mentioned users in your collection of tweets.

In [101]:
# dictionary that keeps track of frequency of words
words_collection = {}
# get number of tweets
number_tweets = len(results['statuses'])

# populate dictionary
for tweet_index in range(number_tweets):
    for word in results['statuses'][tweet_index]['text'].split():
        # if word not in dict then add, else increase count
        if word in words_collection.keys():
            words_collection[word] += 1
        else:
            words_collection[word] = 1

# sort dictionary based on frequency
sorted_results = {k: v for k, v in sorted(words_collection.items(), key=lambda item: item[1], reverse=True)}

# create pretty table
table = PrettyTable(field_names=['word','count'])
for index, word_count_pair in enumerate(sorted_results.items()):
    if index == 30:
        break
    table.add_row(word_count_pair)

print(table)







+--------+-------+
|  word  | count |
+--------+-------+
|   RT   |   58  |
| Sonic  |   37  |
|  the   |   37  |
|   I    |   33  |
|   to   |   24  |
|   a    |   23  |
| sonic  |   22  |
|  and   |   22  |
|   of   |   17  |
|   in   |   13  |
|   so   |   12  |
|  for   |   12  |
|   is   |   12  |
|   my   |   11  |
|  was   |   10  |
| April  |   10  |
| SONIC  |   10  |
|  The   |   10  |
|  with  |   9   |
|   on   |   9   |
| Happy  |   9   |
| movie  |   9   |
|   it   |   8   |
|  not   |   7   |
|   X!   |   6   |
|   7,   |   6   |
| SUMMER |   6   |
|  2019  |   6   |
| Replay |   6   |
|  you   |   6   |
+--------+-------+


Plot a histogram of the number of user mentions in the list using the following bins.

In [ ]:
bins=[0, 10, 20, 30, 40, 50, 100]

# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary






 ** (optional task for fun) Getting "All" friends (followees) and "All" followers of a popular user in the tweets**

* choose a popular twitter user who has many followers in your collection of tweets.
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [35]:
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary







*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this Jupyter notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook.

** How to submit: **

        Please submit through Canvas, in the Assignment "HW1".
        